---
* **Universidad:** UOC (Universitat Oberta de Catalunya)
* **Titulación:** Master Ciencia de Datos/TFM
* **Título:** Segmentación de imágenes utilizando Redes Generativas Antagónicas
* **Actividad**: Segmentación partes cuerpo personas con armas cortas utilizando modelos pre entrenados
* **Autor:** Miguel Ponce
* **Correo:** miguelalejoponce@uoc.edu
* **Fecha:** "Enero, 04 de 2023"
* **Salida**: ipynb
---
## Contexto: Detección de personas

* **Contexto:**
Se desea segmentar las personas y armas cortas con los modelos pre entrenados.

* **Problema:** 
Es necesrio realizar la segmentación de las personas armadas con armas cortas, para tener un conjunto de imagenes que luego serán procesadas para ser utilizadas como conjunto de datos para la red GAN.

* **Solucion :** 
Se utilizara los modelos pre entrenados de las etapas anteriores para generar un nuevo conjunto de datos. Para esto se hara uso de los conjuntos de datos tratado con super pixeles ESRGAN y RDN.
* **Entorno:** con GPU.
  

## Origen del la fuente de datos
* **RDN:** https://github.com/miguelalejo/gan-segmentation-person-gun/blob/main/data/raw/super-resolution-rdn.tar.gz

## Modelos pre entrenados
* **DETECCIÓN ARMA:**
https://drive.google.com/file/d/1xAbxae-WI2uiTPUOZvakSNhvbkNBc2sT/view

* **SEGMENTACIÓN ARMA:** https://drive.google.com/file/d/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob/view

* **SEGMENTACIÓN PERSONA:** https://github.com/de-code/python-tf-bodypix

# Descarga conjunto de datos

In [ ]:
!wget https://media.githubusercontent.com/media/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-rdn.tar.gz  -O super-resolution-rdn.tar.gz 

--2023-01-06 00:22:47--  https://media.githubusercontent.com/media/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-rdn.tar.gz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74354007 (71M) [application/octet-stream]
Saving to: ‘super-resolution-rdn.tar.gz’

super-resolution-rd 100%[===================>]  70.91M  --.-KB/s    in 0.1s    

2023-01-06 00:22:54 (478 MB/s) - ‘super-resolution-rdn.tar.gz’ saved [74354007/74354007]

--2023-01-06 00:22:54--  https://media.githubusercontent.com/media/miguelalejo/gan-segmentation-person-gun/main/data/raw/super-resolution-esrgan-personas-armadas.tar.gz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to

In [ ]:
!tar -xf super-resolution-rdn.tar.gz

# Deteccion Armas Cortas

F## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-j1i86kvj
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-j1i86kvj
  Resolved https://github.com/facebookresearch/detectron2.git to commit 06f2892d2c3125deb207fc61f52484a428d67add
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 712.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 14.4 MB/s eta 0:00:

## Importar Librerias



In [ ]:
import os
import glob
import time
import torch
from pathlib import Path
from PIL import Image

from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_hub as hub
import logging
import torch
from collections import OrderedDict
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.solver import build_lr_scheduler, build_optimizer
from detectron2.checkpoint import DetectionCheckpointer, PeriodicCheckpointer
from detectron2.utils.events import EventStorage
from detectron2.modeling import build_model
import detectron2.utils.comm as comm
from detectron2.engine import default_argument_parser, default_setup, default_writers, launch
from detectron2.data import (
    MetadataCatalog,
    build_detection_test_loader,
    build_detection_train_loader,
)
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    inference_on_dataset,
    print_csv_format,
)

## Mostrar caracteristicas del entorno

In [ ]:
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
torch:  1.13 ; cuda:  cu116
detectron2: 0.6


## Constantes

In [ ]:
# Declaring Constants
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"
TEST_MODE = False
MAX_IMAGES = 100
mainPath = "./"

## Clases


In [ ]:
class DetectImage:
  def __init__(self,sourcePath,outputPath,config,modelName,threshold) -> None:
    self.cfg = config
    print(os.path.join(self.cfg.OUTPUT_DIR, modelName))
    self.cfg.MODEL.WEIGHTS = os.path.join(self.cfg.OUTPUT_DIR, modelName)
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    self.predictor = DefaultPredictor(self.cfg)
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def crop_object(self,imagePath):
    """Crops an object in an image

    Inputs:
      image: PIL image
      box: one box from Detectron2 pred_boxes
    """  
    image = Image.open(imagePath)
    im = cv2.imread(imagePath)
    outputs = self.predictor(im)
    boxes = outputs["instances"].pred_boxes
    # Select 1 box:    
    if list(boxes):
      #print(imagePath)
      #print(boxes,"boxes")
      box = list(boxes)[0].detach().cpu().numpy()         
      x_top_left = box[0]
      y_top_left = box[1]
      x_bottom_right = box[2]
      y_bottom_right = box[3]
      x_center = (x_top_left + x_bottom_right) / 2
      y_center = (y_top_left + y_bottom_right) / 2
      crop_img = image.crop((int(x_top_left), int(y_top_left), int(x_bottom_right), int(y_bottom_right)))
      #plt.figure()
      #plt.axis('off')
      #plt.imshow(crop_img) 
      #plt.show() 
      return True,crop_img,box
    else:
      return False,None,None

  def saveBox(self,box,boxPath):
    with open(boxPath, "w") as f:
      for s in box:
        f.write(str(s) +"\n")
  def readBox(self,boxPath):
    values = []
    with open(boxPath, "r") as f:
      for line in f:
        values.append(float(line.strip()))
    return values
   
  
  def processImages(self):       
    path = self.sourcePath+"/*.jpg"
    print(path)
    images = glob.glob(path)
    index = 0
    start = time.time()
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1             
      hasBox,crop_img,box = self.crop_object(imagePath)
      if hasBox:
        imageName = os.path.basename(imagePath)
        print(imageName)
        name = Path(imagePath).stem        
        boxPath = os.path.join(self.outputPath,"boxes",name+".txt")
        cropImagePath = os.path.join(self.outputPath,"crops",imageName)
        crop_img.save(cropImagePath)
        self.saveBox(box,boxPath)
        print("Time Detect Image : %f" % (time.time() - start))

## Configuracion modelos pre entrenados

In [ ]:
from detectron2.engine import DefaultTrainer
instance = "COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(instance))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(instance)  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## Descaga modelo detección arma pre-entrenado

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1" -O ./output/model_final.pth && rm -rf /tmp/cookies.txt

--2023-01-06 00:26:38--  https://docs.google.com/uc?export=download&confirm=t&id=1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1
Resolving docs.google.com (docs.google.com)... 172.253.118.113, 172.253.118.101, 172.253.118.100, ...
Connecting to docs.google.com (docs.google.com)|172.253.118.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ie1qmbvk75lgqa3k4njtsogtb3darkio/1672964775000/07156903365772177219/*/1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1?e=download&uuid=f6087c9d-b6f6-443c-99d1-35811dcf8100 [following]
--2023-01-06 00:26:39--  https://doc-0g-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ie1qmbvk75lgqa3k4njtsogtb3darkio/1672964775000/07156903365772177219/*/1LNJBizVeyM2541tbb-XD1-JYdcjIfJZ1?e=download&uuid=f6087c9d-b6f6-443c-99d1-35811dcf8100
Resolving doc-0g-bk-docs.googleusercontent.com (doc-0g-bk-docs.googleusercontent.com)... 142.251.12.1

## Creacion proyecto deteccion arma RDN

In [ ]:
!rm -rf guns-detection-rdn
!mkdir guns-detection-rdn
!mkdir "guns-detection-rdn/crops"
!mkdir "guns-detection-rdn/boxes"
!mkdir "guns-detection-rdn/binmask"
!mkdir "guns-detection-rdn/binredmask"

## Creacion Pipeline red pre entrenada detección arma RDN

In [ ]:
sourcePathRDN = os.path.join(mainPath,"super-resolution-rdn/person")
outputPathRDN = os.path.join(mainPath,"guns-detection-rdn")
detectImage = DetectImage(sourcePathRDN,outputPathRDN,cfg,"model_final.pth",0.85)
detectImage.processImages()

./output/model_final.pth
[01/06 00:26:53 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
./super-resolution-rdn/person/*.jpg


/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


gmd_1_frame262_jpg.rf.85e1395cc3f4c0478e725ad3e0a39776.jpg
Time Detect Image : 5.681053
gmd_7_frame31_jpg.rf.c2b756bbb7a32198e03544f73cc19901.jpg
Time Detect Image : 5.967467
youtube_frame182_jpg.rf.9669d3f3c5cbc26c1a3cd2ff0bcf3d7e.jpg
Time Detect Image : 6.177814
448642765-1613827826_jpeg_jpg.rf.6cd6f81336f59b11bbf65bc0b0837824.jpg
Time Detect Image : 6.403949
youtube_frame177_jpg.rf.1b5ea5947a59b8419b30ee62ea27b6e5.jpg
Time Detect Image : 6.613477
armas-320-_jpg.rf.b1ed632d72760d17bd4a6759350f0535.jpg
Time Detect Image : 7.025520
gmd_6_frame15_jpg.rf.fa724e76f63392d665f2d5cc553d4375.jpg
Time Detect Image : 7.314761
gmd_4_frame54_jpg.rf.e7b7f1546c9d247d2dc7fd24ce2a1505.jpg
Time Detect Image : 7.586382
gmd_4_frame75_jpg.rf.13e6cc50266eebd8e70239996a9b6be6.jpg
Time Detect Image : 7.855164
gmd_2_frame72_jpg.rf.04a9ce2601e26802a94c854109dbffb6.jpg
Time Detect Image : 8.088048
gmd_1_frame139_jpg.rf.32893ba6507afe4141ecd24eaf34d5f1.jpg
Time Detect Image : 8.277191
454744887-1615223010_jpeg_

# Segmentacion Armas

## Clases

In [ ]:
class SegmentImage:
  def __init__(self,sourcePath,outputPath,config,modelName,threshold) -> None:
    self.cfg = config
    self.cfg.MODEL.WEIGHTS = os.path.join(self.cfg.OUTPUT_DIR, modelName)
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = threshold
    self.predictor = DefaultPredictor(self.cfg)
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def crop_shape_object(self,imagePath):
    """Crops an object in an image

    Inputs:
      image: PIL image
      box: one box from Detectron2 pred_boxes
    """  
   
    img= cv2.imread(imagePath)
    outputs= self.predictor(img)
    if outputs['instances']: 
      mask= outputs['instances'].get('pred_masks')    
      mask= mask.to('cpu')    
      num, h, w= mask.shape
      bin_mask= np.zeros((h, w))
      index  = 0
      for m in mask:        
          if index > 0:
            break;
          bin_mask = m.numpy().astype(np.uint8)
          #indices = indices.astype(np.uint8)  #convert to an unsigned byte
          bin_mask*=255
          #print(bin_mask.any(0))
          #print(index)
          index = index +1
          #bin_mask.append(m)
      return True,bin_mask
    return False,None
  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePath+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1                
      hasMask,bin_mask = self.crop_shape_object(imagePath)
      if hasMask:
        imageName = os.path.basename(imagePath)
        print(imageName)
        imageNamePng = Path(imageName).stem+'.png'
        im = Image.fromarray(bin_mask)
        imagePngPath = os.path.join(self.outputPath,imageNamePng)                        
        im.save(imagePngPath, "PNG")
        print("Time Segment Image : %f" % (time.time() - start))

## Configuracion modelo segementación pre entrenado

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("asl_poly_train",)#Train dataset registered in a previous cell
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00055
cfg.SOLVER.MAX_ITER = 1200 
cfg.SOLVER.STEPS = []
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  
cfg.TEST.EVAL_PERIOD = 0 # Increase this number if you want to monitor validation performance during training

PATIENCE = 500 #Early stopping will occur after N iterations of no imporovement in total_loss

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## Descaga modelo segemntación arma pre-entrenado

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob" -O ./output/model_final_segmentaion.zip && rm -rf /tmp/cookies.txt

--2023-01-06 00:31:54--  https://docs.google.com/uc?export=download&confirm=t&id=1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob
Resolving docs.google.com (docs.google.com)... 74.125.24.101, 74.125.24.138, 74.125.24.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.24.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9bcqmr4r07i3r20q2knofg8kgvu4v6u6/1672965075000/07156903365772177219/*/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob?e=download&uuid=0a950588-6930-45e7-875a-7252f1f2de47 [following]
--2023-01-06 00:31:54--  https://doc-08-bk-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/9bcqmr4r07i3r20q2knofg8kgvu4v6u6/1672965075000/07156903365772177219/*/1UYza-1NeyJ1SrD94_Kewh9lSb_zOURob?e=download&uuid=0a950588-6930-45e7-875a-7252f1f2de47
Resolving doc-08-bk-docs.googleusercontent.com (doc-08-bk-docs.googleusercontent.com)... 142.251.12.132, 2404

In [ ]:
!unzip -o ./output/model_final_segmentaion.zip -d ./output/

Archive:  ./output/model_final_segmentaion.zip
  inflating: ./output/model_final_segmentaion.pth  


## Creacion Pipeline red pre entrenada segmentación arma RDN

In [ ]:
sourcePathCropsRDN = os.path.join(mainPath,"guns-detection-rdn/crops")
outputPathBinMaskRDN = os.path.join(mainPath,"guns-detection-rdn/binmask")
segmentImage = SegmentImage(sourcePathCropsRDN,outputPathBinMaskRDN,cfg,"model_final_segmentaion.pth",0.60)
segmentImage.processImages()

[01/06 00:32:05 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final_segmentaion.pth ...
gmd_1_frame262_jpg.rf.85e1395cc3f4c0478e725ad3e0a39776.jpg
Time Segment Image : 0.161193
gmd_7_frame31_jpg.rf.c2b756bbb7a32198e03544f73cc19901.jpg
Time Segment Image : 0.298633
youtube_frame182_jpg.rf.9669d3f3c5cbc26c1a3cd2ff0bcf3d7e.jpg
Time Segment Image : 0.409763
448642765-1613827826_jpeg_jpg.rf.6cd6f81336f59b11bbf65bc0b0837824.jpg
Time Segment Image : 0.522700
youtube_frame177_jpg.rf.1b5ea5947a59b8419b30ee62ea27b6e5.jpg
Time Segment Image : 0.632098
armas-320-_jpg.rf.b1ed632d72760d17bd4a6759350f0535.jpg
Time Segment Image : 0.728417
gmd_6_frame15_jpg.rf.fa724e76f63392d665f2d5cc553d4375.jpg
Time Segment Image : 0.828487
gmd_4_frame54_jpg.rf.e7b7f1546c9d247d2dc7fd24ce2a1505.jpg
Time Segment Image : 0.929354
gmd_4_frame75_jpg.rf.13e6cc50266eebd8e70239996a9b6be6.jpg
Time Segment Image : 1.030404
gmd_2_frame72_jpg.rf.04a9ce2601e26802a94c854109dbffb6.jpg
Tim

# Segemtancion partes del cuerpo

## Instalar requisitos

_(Recuerde elegir GPU en tiempo de ejecución si aún no lo ha seleccionado. Tiempo de ejecución --> Cambiar tipo de tiempo de ejecución --> Acelerador de hardware --> GPU)_

In [ ]:
!pip uninstall -y detectron2 
!pip uninstall -y tensorflow
!pip uninstall -y tensorflow_decision_forests
!rm -rf build/ **/*.so

Found existing installation: detectron2 0.6
Uninstalling detectron2-0.6:
  Successfully uninstalled detectron2-0.6
Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Successfully uninstalled tensorflow-2.9.2


In [ ]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 tf_bodypix opencv-python tfjs_graph_converter matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
!python -m pip install --upgrade imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Clases

In [ ]:
class BodyPix:
  def __init__(self,sourcePath,outputPath) -> None:
    #self.bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_FLOAT_50_STRIDE_16))
    self.bodypix_model = load_model(download_model(BodyPixModelPaths.MOBILENET_RESNET50_FLOAT_STRIDE_16))
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def maskImage(self,imagePath):
    imageName = os.path.basename(imagePath)
    
    # load model (once)

    image = tf.keras.utils.load_img(imagePath)
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    # get prediction result
    #image = tf.keras.preprocessing.image.load_img(fullPath)    
    result = self.bodypix_model.predict_single(image_array)

    # simple mask
    mask = result.get_mask(threshold=0.75)
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-mask",imageName),
        mask)

    # colored mask (separate colour for each body part)
    colored_mask = result.get_colored_part_mask(mask)
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-colored-mask",imageName),
        colored_mask)



    poses = result.get_poses()
    image_with_poses = draw_poses(
        image_array.copy(),  # create a copy to ensure we are not modifing the source image
        poses,
        keypoints_color=(255, 100, 100),
        skeleton_color=(100, 100, 255)
    )
    tf.keras.preprocessing.image.save_img(
        os.path.join(self.outputPath,"output-poses",imageName),
        image_with_poses
    )
    

  def processImages(self):       
    start = time.time() 
    path = self.sourcePath+"/*.jpg"
    print(path)
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1
      imageName = os.path.basename(imagePath)
      print(imageName)      
      self.maskImage(imagePath) 
      print("Time Taken Body Pix : %f" % (time.time() - start))

## Liberias BodyPix

In [ ]:
import tensorflow as tf
from tf_bodypix.api import download_model, load_model, BodyPixModelPaths
from tf_bodypix.draw import draw_poses  # utility function using OpenCV

## Creacion proyecto body pix RDN

In [ ]:
!rm -rf body-pix-rdn
!mkdir body-pix-rdn
!mkdir "body-pix-rdn/person"
!mkdir "body-pix-rdn/person/output-mask"
!mkdir "body-pix-rdn/person/output-colored-mask"
!mkdir "body-pix-rdn/person/output-poses"

## Creacion Pipeline red detección partes cuerpo RDN

In [ ]:
sourcePathMask = os.path.join(mainPath,"super-resolution-rdn/person")
outputPathBody = os.path.join(mainPath,"body-pix-rdn/person")
bodyPix = BodyPix(sourcePathMask,outputPathBody)
bodyPix.processImages()

./super-resolution-rdn/person/*.jpg
gmd_1_frame262_jpg.rf.85e1395cc3f4c0478e725ad3e0a39776.jpg
Time Taken Body Pix : 3.464699
gmd_7_frame31_jpg.rf.c2b756bbb7a32198e03544f73cc19901.jpg
Time Taken Body Pix : 4.250197
youtube_frame182_jpg.rf.9669d3f3c5cbc26c1a3cd2ff0bcf3d7e.jpg
Time Taken Body Pix : 5.265009
448642765-1613827826_jpeg_jpg.rf.6cd6f81336f59b11bbf65bc0b0837824.jpg
Time Taken Body Pix : 6.051954
youtube_frame177_jpg.rf.1b5ea5947a59b8419b30ee62ea27b6e5.jpg
Time Taken Body Pix : 7.068708
gmd_2_frame42_jpg.rf.b5a29dc60df7bbe91f7e697185b9d668.jpg
Time Taken Body Pix : 7.819748
armas-320-_jpg.rf.b1ed632d72760d17bd4a6759350f0535.jpg
Time Taken Body Pix : 8.681023
gmd_6_frame15_jpg.rf.fa724e76f63392d665f2d5cc553d4375.jpg
Time Taken Body Pix : 8.783741
gmd_4_frame54_jpg.rf.e7b7f1546c9d247d2dc7fd24ce2a1505.jpg
Time Taken Body Pix : 9.560633
gmd_4_frame75_jpg.rf.13e6cc50266eebd8e70239996a9b6be6.jpg
Time Taken Body Pix : 10.385990
gmd_2_frame72_jpg.rf.04a9ce2601e26802a94c854109dbffb6.jpg

# Reemplazo mascara 

## Librerias

In [ ]:
import shutil

## Clases

In [ ]:
class BitMaskGenerator:
  def __init__(self,sourcePath,outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath

  def createBitMask(self,imagePath):
    img = Image.open(imagePath)
    img = img.convert("RGBA")
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255,0,0, 0))
        else:
            newData.append(item)

    img.putdata(newData)
    return img.convert('RGB')
    

    return False,None
  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePath+"/*.png"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1                      
      rgb_im = self.createBitMask(imagePath)    
      imageName = os.path.basename(imagePath)      
      print(imageName)
      imageNamePng = Path(imageName).stem+'.png'      
      imagePngPath = os.path.join(self.outputPath,imageNamePng) 
      rgb_im.save(imagePngPath)     
      print("Time BitMaskRed Image : %f" % (time.time() - start))

In [ ]:
class ImageBitMaskUtil:
  def __init__(self,sourcePath,pathBitMask, pathBitRedMask, pathBox,outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath
    self.pathBitMask = pathBitMask
    self.pathBox = pathBox
    self.pathBitRedMask = pathBitRedMask
    
  def readBox(self,boxPath):
    values = []
    with open(boxPath, "r") as f:
      for line in f:
        values.append(float(line.strip()))
    return values
  def pasteBitMask(self,pathImage):
    imageName = os.path.basename(pathImage)
    name  = Path(imageName).stem
    pathImageMask = os.path.join(self.pathBitMask,name+".png")
    pathBitRedMask = os.path.join(self.pathBitRedMask,name+".png")
    if os.path.exists(pathImageMask):
      print(imageName)      
      boxImagePath  = os.path.join(self.pathBox,name+'.txt')
      box = self.readBox(boxImagePath)      
      print("box",box)
      with Image.open(pathImage) as imageBody:
          imageBody.load()
      with Image.open(pathBitRedMask) as imgGunRedMask:
          imgGunRedMask.load()
      with Image.open(pathImageMask) as imgGunMask:
          imgGunMask.load()
    
      imageBody.paste(
          imgGunRedMask.resize((imgGunRedMask.width, imgGunRedMask.height)),
          (int(box[0]) ,int(box[1])),
          imgGunMask.resize((imgGunMask.width, imgGunMask.height)),
      )
      pathImageResult = os.path.join(self.outputPath,imageName)
      imageBody.save(pathImageResult)
  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePath+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1               
      imageName = os.path.basename(imagePath)      
      self.pasteBitMask(imagePath)             
      print("Time PasteBitMaskRed into Image : %f" % (time.time() - start))

In [ ]:
class ImageSegmentedUtil:
  def __init__(self,sourcePathSegmented,sourcePath, outputPath) -> None:    
    self.sourcePath = sourcePath
    self.outputPath = outputPath
    self.sourcePathSegmented = sourcePathSegmented    
    


  
  def processImages(self):   
    start = time.time()    
    path = self.sourcePathSegmented+"/*.jpg"    
    images = glob.glob(path)
    index = 0
    for imagePath in images:
      if TEST_MODE:
        if index == MAX_IMAGES:         
          break
        index = index + 1        
      imageName = os.path.basename(imagePath)
      print(imageName)
      originalImagePath = os.path.join(self.sourcePath,imageName) 
      outputImagePath  = os.path.join(self.outputPath,imageName) 
      shutil.copyfile(originalImagePath, outputImagePath)           
      print("Time PasteBitMaskRed into Image : %f" % (time.time() - start))

## Creacion proyecto merge armas segmentadas y personas

In [ ]:
!rm -rf segmented-body-parts-gun-rdn
!mkdir segmented-body-parts-gun-rdn
!mkdir "segmented-body-parts-gun-rdn/original"
!mkdir "segmented-body-parts-gun-rdn/segmentado"
!mkdir "segmented-body-parts-gun-rdn/mascara"

## Creacion Pipeline procesamiento merge partes cuerpo segmentadas y armas

In [ ]:
sourcePatBinMask = os.path.join(mainPath,"guns-detection-rdn/binmask")
outputPathBinRedMask = os.path.join(mainPath,"guns-detection-rdn/binredmask")
bitMaskGenerator = BitMaskGenerator(sourcePatBinMask,outputPathBinRedMask)
bitMaskGenerator.processImages()

images-of-pistol-in-hand_38_jpeg_jpg.rf.64eb6af7c7633b8c51f52226898d3513.png
Time BitMaskRed Image : 0.010542
youtube_frame203_jpg.rf.8c40346005eed33a58ca19b13eb56dff.png
Time BitMaskRed Image : 0.017915
gmd_2_frame35_jpg.rf.72706435636d01ed0900c1bd186dba27.png
Time BitMaskRed Image : 0.020430
youtube_frame194_jpg.rf.708a75013ad0e673b34e0aea03a519a6.png
Time BitMaskRed Image : 0.027231
gmd_1_frame67_jpg.rf.bfb64bb0b775fa62cfbd3a3415236362.png
Time BitMaskRed Image : 0.029977
gmd_2_frame38_jpg.rf.8d3dba4f89d4214bd63a95a0f01787ac.png
Time BitMaskRed Image : 0.031649
mgd_varying_frame42_jpg.rf.3f0b54e698705b8168076ed6358e9123.png
Time BitMaskRed Image : 0.092273
mgd_custom_frame2673_jpg.rf.ffb41b2272c00cfc88df77237ddaa5a3.png
Time BitMaskRed Image : 0.094606
gmd_2_frame69_jpg.rf.330af1c6f284481494d9263c51112348.png
Time BitMaskRed Image : 0.096090
490291863-1624515003_jpeg_jpg.rf.7a7b19b486ca2820ba1d5938cd295ba9.png
Time BitMaskRed Image : 0.105933
mgd_custom_frame959_jpg.rf.923172438c1f3

In [ ]:
sourcePathBody = "body-pix-rdn/person/output-colored-mask"
pathBitMask = "guns-detection-rdn/binmask"
pathBitRedMask = "guns-detection-rdn/binredmask"
boxPath = "guns-detection-rdn/boxes"
outputPathSegmented = "segmented-body-parts-gun-rdn/segmentado"
imageBitMaskUtil = ImageBitMaskUtil(sourcePathBody,pathBitMask, pathBitRedMask, boxPath,outputPathSegmented)
imageBitMaskUtil.processImages()

gmd_1_frame262_jpg.rf.85e1395cc3f4c0478e725ad3e0a39776.jpg
box [13.870683, 263.45926, 81.11342, 333.8316]
Time PasteBitMaskRed into Image : 0.016909
gmd_7_frame31_jpg.rf.c2b756bbb7a32198e03544f73cc19901.jpg
box [9.361863, 262.9076, 68.840576, 331.826]
Time PasteBitMaskRed into Image : 0.029462
youtube_frame182_jpg.rf.9669d3f3c5cbc26c1a3cd2ff0bcf3d7e.jpg
box [25.73595, 683.5022, 173.37515, 871.8126]
Time PasteBitMaskRed into Image : 0.044188
448642765-1613827826_jpeg_jpg.rf.6cd6f81336f59b11bbf65bc0b0837824.jpg
box [29.54971, 436.31122, 74.006805, 534.99176]
Time PasteBitMaskRed into Image : 0.056291
youtube_frame177_jpg.rf.1b5ea5947a59b8419b30ee62ea27b6e5.jpg
box [233.31496, 678.60925, 367.69418, 842.69495]
Time PasteBitMaskRed into Image : 0.071368
Time PasteBitMaskRed into Image : 0.071814
armas-320-_jpg.rf.b1ed632d72760d17bd4a6759350f0535.jpg
box [334.38138, 169.93633, 543.57635, 377.89304]
Time PasteBitMaskRed into Image : 0.091492
gmd_6_frame15_jpg.rf.fa724e76f63392d665f2d5cc553d43

In [ ]:
sourcePathBody = "body-pix-rdn/person/output-mask"
pathBitMask = "guns-detection-rdn/binmask"
pathBitRedMask = "guns-detection-rdn/binmask"
boxPath = "guns-detection-rdn/boxes"
outputPathSegmented = "segmented-body-parts-gun-rdn/mascara"
imageBitMaskUtil = ImageBitMaskUtil(sourcePathBody,pathBitMask, pathBitRedMask, boxPath,outputPathSegmented)
imageBitMaskUtil.processImages()

gmd_1_frame262_jpg.rf.85e1395cc3f4c0478e725ad3e0a39776.jpg
box [13.870683, 263.45926, 81.11342, 333.8316]
Time PasteBitMaskRed into Image : 0.009794
gmd_7_frame31_jpg.rf.c2b756bbb7a32198e03544f73cc19901.jpg
box [9.361863, 262.9076, 68.840576, 331.826]
Time PasteBitMaskRed into Image : 0.016481
youtube_frame182_jpg.rf.9669d3f3c5cbc26c1a3cd2ff0bcf3d7e.jpg
box [25.73595, 683.5022, 173.37515, 871.8126]
Time PasteBitMaskRed into Image : 0.023473
448642765-1613827826_jpeg_jpg.rf.6cd6f81336f59b11bbf65bc0b0837824.jpg
box [29.54971, 436.31122, 74.006805, 534.99176]
Time PasteBitMaskRed into Image : 0.029766
youtube_frame177_jpg.rf.1b5ea5947a59b8419b30ee62ea27b6e5.jpg
box [233.31496, 678.60925, 367.69418, 842.69495]
Time PasteBitMaskRed into Image : 0.036938
Time PasteBitMaskRed into Image : 0.037034
armas-320-_jpg.rf.b1ed632d72760d17bd4a6759350f0535.jpg
box [334.38138, 169.93633, 543.57635, 377.89304]
Time PasteBitMaskRed into Image : 0.046276
gmd_6_frame15_jpg.rf.fa724e76f63392d665f2d5cc553d43

In [ ]:
outputOriginalPath = "segmented-body-parts-gun-rdn/original"
imageSegmentedUtil = ImageSegmentedUtil(outputPathSegmented,sourcePathMask, outputOriginalPath)
imageSegmentedUtil.processImages()

gmd_1_frame262_jpg.rf.85e1395cc3f4c0478e725ad3e0a39776.jpg
Time PasteBitMaskRed into Image : 0.004039
gmd_7_frame31_jpg.rf.c2b756bbb7a32198e03544f73cc19901.jpg
Time PasteBitMaskRed into Image : 0.004718
youtube_frame182_jpg.rf.9669d3f3c5cbc26c1a3cd2ff0bcf3d7e.jpg
Time PasteBitMaskRed into Image : 0.005405
448642765-1613827826_jpeg_jpg.rf.6cd6f81336f59b11bbf65bc0b0837824.jpg
Time PasteBitMaskRed into Image : 0.006128
youtube_frame177_jpg.rf.1b5ea5947a59b8419b30ee62ea27b6e5.jpg
Time PasteBitMaskRed into Image : 0.006798
armas-320-_jpg.rf.b1ed632d72760d17bd4a6759350f0535.jpg
Time PasteBitMaskRed into Image : 0.007477
gmd_6_frame15_jpg.rf.fa724e76f63392d665f2d5cc553d4375.jpg
Time PasteBitMaskRed into Image : 0.008148
gmd_4_frame54_jpg.rf.e7b7f1546c9d247d2dc7fd24ce2a1505.jpg
Time PasteBitMaskRed into Image : 0.008913
gmd_4_frame75_jpg.rf.13e6cc50266eebd8e70239996a9b6be6.jpg
Time PasteBitMaskRed into Image : 0.009646
gmd_2_frame72_jpg.rf.04a9ce2601e26802a94c854109dbffb6.jpg
Time PasteBitMask

## Compresion Archivos

In [ ]:
!rm -rf segmented-body-parts-gun-rdn.tar.gz
!tar -czvf segmented-body-parts-gun-rdn.tar.gz "./segmented-body-parts-gun-rdn"

./segmented-body-parts-gun-rdn/
./segmented-body-parts-gun-rdn/mascara/
./segmented-body-parts-gun-rdn/mascara/gmd_1_frame262_jpg.rf.85e1395cc3f4c0478e725ad3e0a39776.jpg
./segmented-body-parts-gun-rdn/mascara/gmd_7_frame31_jpg.rf.c2b756bbb7a32198e03544f73cc19901.jpg
./segmented-body-parts-gun-rdn/mascara/youtube_frame182_jpg.rf.9669d3f3c5cbc26c1a3cd2ff0bcf3d7e.jpg
./segmented-body-parts-gun-rdn/mascara/448642765-1613827826_jpeg_jpg.rf.6cd6f81336f59b11bbf65bc0b0837824.jpg
./segmented-body-parts-gun-rdn/mascara/youtube_frame177_jpg.rf.1b5ea5947a59b8419b30ee62ea27b6e5.jpg
./segmented-body-parts-gun-rdn/mascara/armas-320-_jpg.rf.b1ed632d72760d17bd4a6759350f0535.jpg
./segmented-body-parts-gun-rdn/mascara/gmd_6_frame15_jpg.rf.fa724e76f63392d665f2d5cc553d4375.jpg
./segmented-body-parts-gun-rdn/mascara/gmd_4_frame54_jpg.rf.e7b7f1546c9d247d2dc7fd24ce2a1505.jpg
./segmented-body-parts-gun-rdn/mascara/gmd_4_frame75_jpg.rf.13e6cc50266eebd8e70239996a9b6be6.jpg
./segmented-body-parts-gun-rdn/mascara/

## Cargar en drive resultados

In [ ]:
import os
from google.colab import drive
ROOT_FOLDER='/content/'
ROOT_DRIVE_FOLDER=ROOT_FOLDER+'drive/'
drive.mount(ROOT_DRIVE_FOLDER)

Mounted at /content/drive/


In [ ]:
!cp --suffix=_v_$(date +"%y-%m-%d--%T")  ./segmented-body-parts-gun-rdn.tar.gz /content/drive/MyDrive/tmf-person-data/data_set_processed/segmented-body-parts-gun-rdn